## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
# Run only once, to solve the conflict with ROS
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [2]:
import cv2
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib qt

# prepare object points
nx = 9 # TODO: enter the number of inside corners in x
ny = 6 # TODO: enter the number of inside corners in y

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(50)

cv2.destroyAllWindows() 

## And so on and so forth...

## 2. Apply a distortion correction to raw images.

In [3]:
# Calculate the distortion function
# Write a function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## 3. Use color transforms, gradients, etc., to create a thresholded binary image. 

In [30]:
def gradients_threshold(grad_channel, ksize=15, mag_thresh=(30, 255), dir_thresh=(0.7, 1.3)):
    
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    abs_sobel_x = np.absolute(cv2.Sobel(grad_channel, cv2.CV_64F, 1, 0))
    abs_sobel_y = np.absolute(cv2.Sobel(grad_channel, cv2.CV_64F, 0, 1))
    

    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(abs_sobel_x**2 + abs_sobel_y**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(abs_sobel_x, abs_sobel_y)
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1

    # Combine the pixels where: 
    #     both the xx and yy gradients meet the threshold criteria
    #   or 
    #     the gradient magnitude and direction are both within their threshold values
    combined = np.zeros_like(mag_binary)
    combined[(mag_binary == 1) & (dir_binary == 1)] = 1
    
    '''
    # Plotting images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('magnitude', fontsize=50)
    ax1.imshow(mag_binary, cmap='gray')
    ax2.set_title('direction', fontsize=50)
    ax2.imshow(dir_binary, cmap='gray')
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    '''   
    
    return combined


# Gradients & Color threshold combined function
def combined_threshold(img, ksize=15, mag_thresh=(30, 255), dir_thresh=(0.7, 1.3), l_thresh=(120, 255), s_thresh=(170, 255)):
    # Choose a Sobel kernel size
    # Choose a larger odd number to smooth gradient measurements
    
    # Convert to HLS color space and separate the L and S channel
    # l_channel = img[:,:,0]
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Threshold l channel
    scale_factor = np.max(l_channel)/255 
    l_channel = (l_channel/scale_factor).astype(np.uint8) 
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    # Threshold s channel
    scale_factor = np.max(s_channel)/255
    s_channel = (s_channel/scale_factor).astype(np.uint8)
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Select l channel to apply gradients thresholding
    l_grad_threshed = gradients_threshold(l_binary, ksize, mag_thresh, dir_thresh) # x_thresh, y_thresh,
    # Select s channel to apply gradients thresholding
    s_grad_threshed = gradients_threshold(s_binary, ksize, mag_thresh, dir_thresh) # x_thresh, y_thresh,
    
    # Stack each channel
    combined_ls = np.zeros_like(s_channel)
    combined_ls[(s_grad_threshed == 1) | (l_grad_threshed == 1)] = 1
    combined_color = np.dstack(( np.zeros_like(l_grad_threshed), l_grad_threshed, s_grad_threshed )) * 255
    
    '''
    # Plotting images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('L', fontsize=50)
    ax1.imshow(l_binary, cmap='gray')
    ax2.set_title('S', fontsize=50)
    ax2.imshow(s_binary, cmap='gray')
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    '''
    
    
    return combined_ls, combined_color

## 4. Apply a perspective transform to rectify binary image ("birds-eye view").

In [5]:
def perspective_transform(img, M, imshape):
    
    transformed = cv2.warpPerspective(img, M, imshape)
    
    return transformed


def calculate_transform_matrix(imshape, left, right, top, bot, top_left, new_left):
    
    src_left_top = (top_left, top)
    src_right_top = (imshape[0] - top_left, top)
    src_right_bot = (right, bot)
    src_left_bot = (left, bot)
    src = np.float32([[src_left_top, src_right_top, src_right_bot, src_left_bot]])
    new_right = imshape[0] - new_left
    dst_left_top = (new_left, 0)
    dst_right_top = (new_right, 0)
    dst_right_bot = (new_right, imshape[1])
    dst_left_bot = (new_left, imshape[1])
    dst = np.float32([[dst_left_top, dst_right_top, dst_right_bot, dst_left_bot]])
    
    # calculate transform and inverse transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    return M, Minv


def calculate_center_margin(imshape, new_left):
    
    center_margin = int( 0.5*imshape[0] - new_left )
    
    return center_margin

## 5. Detect lane pixels and fit to find the lane boundary.

In [6]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        # (win_xleft_high,win_y_high),(0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),
        # (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped, center_margin):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        center_fitx = (left_fitx + right_fitx)/2
        center_fit = np.polyfit(ploty, center_fitx, 2)
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    # out_img[lefty, leftx] = [255, 0, 0]
    # out_img[righty, rightx] = [0, 0, 255]

    # Create fitted lane points
    left_fit_pts = np.vstack((left_fitx, ploty)).astype(np.int32).T
    right_fit_pts = np.vstack((right_fitx, ploty)).astype(np.int32).T
    center_fit_pts = np.vstack((center_fitx, ploty)).astype(np.int32).T
    
    # Draw the fitted lane points
    fit_thickness = int( binary_warped.shape[1]/15 )
    cv2.polylines(out_img,  [left_fit_pts],  False,  (0, 0, 100),  fit_thickness)
    cv2.polylines(out_img,  [right_fit_pts],  False,  (100, 0, 0),  fit_thickness)
    cv2.polylines(out_img,  [center_fit_pts],  False,  (0, 100, 0),  2*center_margin)
    ## End visualization steps ##
    
    return out_img, center_fit, left_fit, right_fit 


## 5. b) Convolution method

In [34]:
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output


def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
	    # convolve the window into the vertical slice of the image
	    image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
	    conv_signal = np.convolve(window, image_layer)
	    # Find the best left centroid by using past left center as a reference
	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
	    offset = window_width/2
	    l_min_index = int(max(l_center+offset-margin,0))
	    l_max_index = int(min(l_center+offset+margin,image.shape[1]))
	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
	    # Find the best right centroid by using past right center as a reference
	    r_min_index = int(max(r_center+offset-margin,0))
	    r_max_index = int(min(r_center+offset+margin,image.shape[1]))
	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
	    # Add what we found for that layer
	    window_centroids.append((l_center,r_center))

    return window_centroids


def convolution_window(warped, center_margin):
    
    # window settings
    window_width = 30 
    window_height = 90 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching
    
    imshape = (warped.shape[1], warped.shape[0])
    out_img = np.dstack((warped, warped, warped))
    
    # window_centroids contains (l_center,r_center), from bottom
    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        leftx = []
        rightx = []
        lefty = []
        righty = []
        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            leftx.append(window_centroids[level][0])
            rightx.append(window_centroids[level][1])
            lefty.append(imshape[1] - level*window_height - window_height/2)
            righty.append(imshape[1] - level*window_height - window_height/2)
            
        # Fit a second order polynomial to each using `np.polyfit`
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, imshape[1]-1, imshape[1] )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
            center_fitx = (left_fitx + right_fitx)/2
            center_fit = np.polyfit(ploty, center_fitx, 2)
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
            
        # Create fitted lane points
        left_fit_pts = np.vstack((left_fitx, ploty)).astype(np.int32).T
        right_fit_pts = np.vstack((right_fitx, ploty)).astype(np.int32).T
        center_fit_pts = np.vstack((center_fitx, ploty)).astype(np.int32).T
    
        # Draw the fitted lane points
        fit_thickness = int( imshape[0]/15 )
        cv2.polylines(out_img,  [left_fit_pts],  False,  (0, 0, 100),  fit_thickness)
        cv2.polylines(out_img,  [right_fit_pts],  False,  (100, 0, 0),  fit_thickness)
        cv2.polylines(out_img,  [center_fit_pts],  False,  (0, 100, 0),  2*center_margin)
        ## End visualization steps ## 
        
    # If no window centers found, just display orginal road image
    else:
        out_img = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    
    return out_img, center_fit, left_fit, right_fit 


### Once lane lines are found, only search around within a margin

In [ ]:
# Search around within the margin
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, center_margin, left_fit=[0, 0, 0], right_fit=[0, 0, 0]):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    if( (left_fit != [0, 0, 0]) | (right_fit != [0, 0, 0]) ):
        margin = 100
    
        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
    
        ### TO-DO: Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###
        ### Hint: consider the window areas for the similarly named variables ###
        ### in the previous quiz, but change the windows to our new search area ###
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                                       left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                                                                             left_fit[1]*nonzeroy + left_fit[2] + margin)))
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                                        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                                                                               right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit new polynomials
        left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
        # Fit the left and right
        left_fit = np.polyfit(ploty, left_fitx, 2)
        right_fit = np.polyfit(ploty, right_fitx, 2)
        # Fit the center curverture
        center_fitx = (left_fitx + right_fitx)/2
        center_fit = np.polyfit(ploty, center_fitx, 2)
    
    
        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)) # *255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        # out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        # out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
        # Generate a polygon to illustrate the safe driving area
        center_line_window1 = np.array([np.transpose(np.vstack([center_fitx-center_margin, ploty]))])
        center_line_window2 = np.array([np.flipud(np.transpose(np.vstack([center_fitx+center_margin, ploty])))])
        center_line_pts = np.hstack((center_line_window1, center_line_window2))
    
    
        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 0, 100))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (100, 0, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        cv2.fillPoly(window_img, np.int_([center_line_pts]), (0, 100, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        
        # Plot the polynomial lines onto the image
        # plt.plot(left_fitx, ploty, color='yellow')
        # plt.plot(right_fitx, ploty, color='yellow')
        # plt.plot(center_fitx, ploty, color='red')
        ## End visualization steps ##
        
    else:
        result, center_fit, left_fit, right_fit = convolution_window(binary_warped, center_margin)
        # result, center_fit, left_fit, right_fit = fit_polynomial(binary_warped, center_margin)
    
    return result, center_fit, left_fit, right_fit 

## 6. Determine the curvature of the lane and vehicle position with respect to center.

In [36]:
def calculate_distance2pixel(center_margin, imshape):
    
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = 3.7/2/center_margin # meters per pixel in x dimension
    ym_per_pix = 37/imshape[1] # meters per pixel in y dimension
    
    return xm_per_pix, ym_per_pix


def measure_curvature_real(imshape, center_margin, center_fit):
    
    # Calculates the curvature of polynomial functions in meters.
    xm_per_pix, ym_per_pix = calculate_distance2pixel(center_margin, imshape)
    
    center_fit_cr = [center_fit[0]*xm_per_pix/ym_per_pix/ym_per_pix, 
                     center_fit[1]*xm_per_pix/ym_per_pix, center_fit[2]*xm_per_pix]
    
    # Define y-value where we want radius of curvature
    ploty = np.linspace(0, imshape[1]-1, num=imshape[1])# to cover same y-range as image
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    center_curverad = ((1 + (2*center_fit_cr[0]*y_eval*ym_per_pix + center_fit_cr[1])**2)**1.5) / np.absolute(2*center_fit_cr[0])
    lane_center = center_fit[0]*imshape[1]**2 + center_fit[1]*imshape[1] + center_fit[2]
    track_error = (0.5*imshape[0] - lane_center) * xm_per_pix *100 # in centimeter
    if(track_error >= 0):
        position_flag = "right"
    else:
        position_flag = "left"
    abs_track_error = np.abs(track_error)
    
    return center_curverad, abs_track_error, position_flag

## 7. Warp the detected lane boundaries back onto the original image. 

With the help of the `perspective_transform()` function in step 4

In [37]:
def display_lane(raw_img, result, Minv, imshape):
    
    result_unwarped = perspective_transform(result, Minv, imshape)
    # result_unwarped = cv2.cvtColor(result_unwarped, cv2.COLOR_BGR2RGB)
    output_image = cv2.addWeighted(raw_img, 1, result_unwarped, 1, 0)
    
    return output_image

## 8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [38]:
def add_text(img, center_curverad, abs_track_error, position_flag):
    center_curverad = str(np.round(center_curverad, 1))
    abs_track_error = str(np.round(abs_track_error, 1))
    text_1 = "Radius of Lane = " + center_curverad + "m"
    text_2 = "Vehicle is " + abs_track_error + "cm " + position_flag + " of center"
    cv2.putText(img, text_1, (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    cv2.putText(img, text_2, (200, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    
    return img

## 9. Final Pipeline

### A small debug function to help show the images during the process 

In [39]:
def debug(combined_threshed, binary_warped, result, output_image):
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Lane Line Image', fontsize=50)
    ax1.imshow(combined_threshed)
    ax2.set_title('Output Image', fontsize=50)
    ax2.imshow(binary_warped, cmap='gray')
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    
    # Plotting images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Lane Line Image', fontsize=50)
    ax1.imshow(result)
    ax2.set_title('Output Image', fontsize=50)
    ax2.imshow(output_image)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    
    return

In [40]:
# A seperate cell to initialise left and right fit parameters
# left_fit = np.array([2.13935315e-04, -3.77507980e-01,  4.76902175e+02])
# right_fit = np.array([4.17622148e-04, -4.93848953e-01,  1.11806170e+03])
left_fit = [0, 0, 0]
right_fit = [0, 0, 0]

In [41]:
# Testing with the sliding windows method
def img_pipeline(raw_img):
    
    '''1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.'''
    '''2. Apply a distortion correction to raw images.'''
    undistorted = cal_undistort(raw_img, objpoints, imgpoints)
    imshape = (undistorted.shape[1], undistorted.shape[0])
    
    
    '''3. Use color transforms, gradients, etc., to create a thresholded binary image.'''
    ksize = 15
    mag_thresh = (100, 255) # (150, 255) (100,255)
    dir_thresh = (0.7, 1.3) # (0.7, 1.3)
    l_thresh = (50, 130) #l (50, 130) #h (15, 100)
    s_thresh = (130, 255) # (100, 255) (130, 255) (170, 255)
    # Apply the threshholds
    combined_threshed, combined_color = combined_threshold(undistorted, ksize, mag_thresh, dir_thresh, l_thresh, s_thresh)

    
    '''4. Apply a perspective transform to rectify binary image ("birds-eye view").'''
    # Appoint source points and destination points
    left = int(0.2*imshape[0])
    right = int(0.8*imshape[0])
    top = int(0.65*imshape[1])
    bot = int(0.95*imshape[1])
    top_left = int(0.445*imshape[0])
    new_left = 250 # 300
    # Given src and dst points, calculate the perspective transform matrix
    M, Minv = calculate_transform_matrix(imshape, left, right, top, bot, top_left, new_left)
    # Warp the image to top-down view
    binary_warped = perspective_transform(combined_threshed, M, imshape)
    
    
    '''5. Detect lane pixels and fit to find the lane boundary.'''
    # Calculate half of the lane width in pixels
    center_margin = calculate_center_margin(imshape, new_left)
    
    # a) Sliding windows method
    # result, center_fit, left_fit, right_fit = fit_polynomial(binary_warped, center_margin)
    # b) Convolution window method
    # result, center_fit, left_fit, right_fit = convolution_window(binary_warped, center_margin)
    # Polynomial fit values from the previous frame 
    result, center_fit, left_fit, right_fit = search_around_poly(binary_warped, center_margin)

    
    '''6. Determine the curvature of the lane and vehicle position with respect to center.'''
    # Calculate the radius of curvature in meters at the center, the error in position and left/right flag
    center_curverad, abs_track_error, position_flag = measure_curvature_real(imshape, center_margin, center_fit)
    
    '''7. Warp the detected lane boundaries back onto the original image.'''
    output_image = display_lane(raw_img, result, Minv, imshape)
    
    '''8. Output visual display of the lane boundaries'''
    # and numerical estimation of lane curvature and vehicle position.
    output_image= add_text(output_image, center_curverad, abs_track_error, position_flag)
    
    # Plotting images for debugging
    # debug(combined_color, binary_warped, result, output_image)
    
    return output_image

# Test on images

In [42]:
for x in os.listdir("test_images/"):
    input_path = "test_images/" + x
    if os.path.isdir(input_path):
        continue
    
    # Read in each raw image
    raw_img = cv2.imread(input_path) 
    
    %time output_img = img_pipeline(raw_img)
    
    # Outout the image to directory "/test_image_output"
    output_path = "test_image_output/" + x
    if os.path.isdir(output_path):
        continue
    cv2.imwrite(output_path, output_img)


CPU times: user 1.42 s, sys: 231 µs, total: 1.42 s
Wall time: 850 ms
CPU times: user 2.11 s, sys: 15.5 ms, total: 2.12 s
Wall time: 886 ms
CPU times: user 2.08 s, sys: 28.1 ms, total: 2.11 s
Wall time: 871 ms
CPU times: user 2.16 s, sys: 31.5 ms, total: 2.19 s
Wall time: 912 ms
CPU times: user 2.13 s, sys: 16.7 ms, total: 2.15 s
Wall time: 917 ms
CPU times: user 2.1 s, sys: 16.2 ms, total: 2.11 s
Wall time: 899 ms
CPU times: user 2.13 s, sys: 24.8 ms, total: 2.15 s
Wall time: 894 ms
CPU times: user 2.17 s, sys: 19.7 ms, total: 2.19 s
Wall time: 918 ms
CPU times: user 2.07 s, sys: 23.8 ms, total: 2.09 s
Wall time: 941 ms
CPU times: user 2.13 s, sys: 11.8 ms, total: 2.14 s
Wall time: 965 ms
CPU times: user 2.14 s, sys: 32.1 ms, total: 2.17 s
Wall time: 899 ms
CPU times: user 2.12 s, sys: 28.1 ms, total: 2.15 s
Wall time: 895 ms
CPU times: user 2.16 s, sys: 4.1 ms, total: 2.16 s
Wall time: 903 ms


# Runing on Videos

In [17]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## a) Project Video

In [47]:
# Set input and output paths
input_path_1 = "project_video.mp4"
output_path_1 = 'output_videos/' + input_path_1

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip_1 = VideoFileClip(input_path_1)
clip_1_output = clip_1.fl_image(img_pipeline) # NOTE: this function expects color images!!
%time clip_1_output.write_videofile(output_path_1, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4




t:  10%|▉         | 124/1260 [02:21<18:45,  1.01it/s, now=None]


t:  20%|█▉        | 246/1260 [04:20<16:30,  1.02it/s, now=None]


t:  29%|██▉       | 368/1260 [06:20<14:22,  1.03it/s, now=None]


...

t:  78%|███████▊  | 978/1260 [16:44<04:36,  1.02it/s, now=None]


t:  87%|████████▋ | 1098/1260 [18:42<02:38,  1.02it/s, now=None]


t:  97%|█████████▋| 1218/1260 [20:40<00:41,  1.01it/s, now=None]


t:   0%|          | 3/1260 [33:16<12:14,  1.71it/s, now=None]   

Moviepy - Done !
Moviepy - video ready output_videos/project_video.mp4
CPU times: user 49min 51s, sys: 1min 7s, total: 50min 58s
Wall time: 21min 22s


In [49]:
# Visualization
HTML("""
<video width="800" height="450" controls>
  <source src="{0}">
</video>
""".format(output_path_1))

## b) Challenge Video

In [45]:
# Set input and output paths
input_path_2 = "challenge_video.mp4"
output_path_2 = 'output_videos/' + input_path_2
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip = VideoFileClip(input_path_2)
clip_output = clip.fl_image(img_pipeline) # NOTE: this function expects color images!!
%time clip_output.write_videofile(output_path_2, audio=False)

t:   0%|          | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/challenge_video.mp4.
Moviepy - Writing video output_videos/challenge_video.mp4




t:  26%|██▌       | 126/485 [01:54<05:49,  1.03it/s, now=None]


t:  52%|█████▏    | 250/485 [03:55<03:50,  1.02it/s, now=None]


t:  77%|███████▋  | 374/485 [05:58<01:54,  1.04s/it, now=None]


t:   0%|          | 3/1260 [08:08<12:14,  1.71it/s, now=None] 

Moviepy - Done !
Moviepy - video ready output_videos/challenge_video.mp4
CPU times: user 18min 43s, sys: 21.3 s, total: 19min 5s
Wall time: 7min 54s


In [46]:
# Visualization
HTML("""
<video width="800" height="450" controls>
  <source src="{0}">
</video>
""".format(output_path_2))

## c) Harder Challenge Video

In [50]:
# Set input and output paths
input_path_3 = "harder_challenge_video.mp4"
output_path_3 = 'output_videos/' + input_path_3
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip = VideoFileClip(input_path_3)
clip_output = clip.fl_image(img_pipeline) # NOTE: this function expects color images!!
%time clip_output.write_videofile(output_path_3, audio=False)

t:   0%|          | 0/1199 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/harder_challenge_video.mp4.
Moviepy - Writing video output_videos/harder_challenge_video.mp4




t:  10%|█         | 124/1199 [02:00<17:41,  1.01it/s, now=None]


t:  21%|██        | 246/1199 [04:00<15:27,  1.03it/s, now=None]


t:  31%|███       | 368/1199 [06:00<13:35,  1.02it/s, now=None]


...

t:  71%|███████▏  | 856/1199 [13:58<05:38,  1.01it/s, now=None]


t:  82%|████████▏ | 978/1199 [15:57<03:34,  1.03it/s, now=None]


t:  92%|█████████▏| 1098/1199 [17:54<01:39,  1.02it/s, now=None]


t:   0%|          | 3/1260 [1:08:41<12:14,  1.71it/s, now=None] 

Moviepy - Done !
Moviepy - video ready output_videos/harder_challenge_video.mp4
CPU times: user 46min 41s, sys: 1min 2s, total: 47min 44s
Wall time: 19min 35s


In [51]:
# Visualization
HTML("""
<video width="800" height="450" controls>
  <source src="{0}">
</video>
""".format(output_path_3))